# 2. Data Cleaning & Merging

In [22]:
import pandas as pd

skiresortinfo_df = pd.read_csv('skiresort_dot_info_updated.csv') # Updated version from the web_scraping notebook
stormskiing_df = pd.read_csv('stormskiing.csv') # Data from stormskiing.com

### Merging data from different sources

In [2]:
# Ensure name columns are stripped for cleaner comparison
skiresortinfo_df['name'] = skiresortinfo_df['name'].str.strip()
stormskiing_df['name'] = stormskiing_df['name'].str.strip()

print(f'Number of areas from skiresort.info: {len(skiresortinfo_df)}')
print(f'Number of areas from stormskiing.com: {len(stormskiing_df)}')

# Create a set of exact matches
exact_matches = set(skiresortinfo_df['name']) & set(stormskiing_df['name'])

# Initialize an empty list to store confirmed merged rows
merged_rows = []

for name in sorted(exact_matches):
    info_row = skiresortinfo_df[skiresortinfo_df['name'] == name].iloc[0]
    storm_row = stormskiing_df[stormskiing_df['name'] == name].iloc[0]
    
    merged = pd.concat([info_row, storm_row.drop(labels='name')])
    merged_rows.append(merged)

# Combine all confirmed merged rows into a DataFrame
if merged_rows:
    resort_info = pd.DataFrame(merged_rows)
    print(f"Merged DataFrame successfully with {len(resort_info)} matches.")
else:
    print("No rows merged.")

Number of areas from skiresort.info: 455
Number of areas from stormskiing.com: 455
Merged DataFrame successfully with 455 matches.


In [3]:
# Renaming a few mountains and removing irrelevant mountains (private, lessons only, etc.)

resort_info.loc[resort_info['name'] == 'Fort Wainwright/\u200bBirch Hill', 'name'] = 'Fort Wainwright – Birch Hill'
resort_info.loc[resort_info['name'] == 'Big Squaw', 'name'] = 'Big Moose'
resort_info = resort_info[resort_info['name'] != 'Challenge Mountain'] # Private
resort_info = resort_info[resort_info['name'] != 'Cockaigne'] # Closed for good
resort_info = resort_info[resort_info['name'] != 'Deer Mountain'] # Recently went private
resort_info = resort_info[resort_info['name'] != 'Eagle Rock'] # Private
resort_info = resort_info[resort_info['name'] != 'Fox Hill'] # Private
resort_info = resort_info[resort_info['name'] != 'Haystack Mountain – The Hermitage Club'] # Private
resort_info = resort_info[resort_info['name'] != 'Heiliger Huegel Ski Club'] # Private
resort_info = resort_info[resort_info['name'] != 'Mount Greylock Ski Club'] # Private
resort_info = resort_info[resort_info['name'] != 'Rocking Horse Ranch'] # Only open to guests staying at the ranch
resort_info = resort_info[resort_info['name'] != 'Sahalie Ski Club'] # Private
resort_info = resort_info[resort_info['name'] != 'Sky Tavern'] # Skiing co-op primarily focused on lessons
resort_info = resort_info[resort_info['name'] != 'Utah Olympic Park'] # Private Olympic training facility
resort_info = resort_info[resort_info['name'] != 'Wasatch Peaks Ranch'] # Private
resort_info = resort_info[resort_info['name'] != 'Woodward Park City'] # Made for freestyle training with no regular trails
resort_info = resort_info[resort_info['name'] != 'Yellowstone Club'] # Private

In [4]:
lift_df = pd.read_csv('lift_data.csv')
resort_info.drop(columns=['num_lifts'], inplace=True)

print(f'Number of areas from resort_info: {len(resort_info)}')
print(f'Number of areas from lift_data: {len(lift_df)}')

# Create a set of exact matches
exact_matches = set(resort_info['name']) & set(lift_df['name'])

# Initialize an empty list to store confirmed merged rows
merged_rows = []

for name in sorted(exact_matches):
    info_row = resort_info[resort_info['name'] == name].iloc[0]
    lift_row = lift_df[lift_df['name'] == name].iloc[0]
    
    merged = pd.concat([info_row, lift_row.drop(labels='name')])
    merged_rows.append(merged)

# Combine all confirmed merged rows into a DataFrame
if merged_rows:
    resort_info = pd.DataFrame(merged_rows)
    print(f"Merged DataFrame successfully with {len(resort_info)} matches.")
else:
    print("No rows merged.")

Number of areas from resort_info: 440
Number of areas from lift_data: 440
Merged DataFrame successfully with 440 matches.


In [5]:
# Fill the NaN values in the lift data
columns_to_fill = ['Tram', 'Gondola', 'Pulse Gondola', 'Cabriolet', '8-pack', '6-pack', 'Chondola', 'Detach Quad', 'Fixed Quad',
                   'Triple', 'Detach Triple', 'Double', 'Single', 'T-Bar', 'J-Bar', 'Poma/platter', 'Ropetow', 'Carpet']
resort_info[columns_to_fill] = resort_info[columns_to_fill].fillna(0).astype(int)

In [6]:
resort_info

,name,url,price,base_elevation_ft,peak_elevation_ft,vertical_drop_ft,slope_mi,state,skiable_acres,annual_snowfall,...,Detach Triple,Double,Single,T-Bar,J-Bar,Poma/platter,Ropetow,Carpet,total_lifts,surface_lifts_only
0,49 Degrees North,https://www.skiresort.info/ski-resort/49-degre...,84.0,3924.0,5774.0,1850.0,42.0,Washington,2325.0,301.0,...,0,4,0,0,0,0,0,1,7,No
1,Abenaki,https://www.skiresort.info/ski-resort/abenaki/,20.0,591.0,820.0,230.0,1.0,New Hampshire,18.0,NaN,...,0,0,0,0,0,0,2,0,2,Yes
2,Adventure Mountain,https://www.skiresort.info/ski-resort/adventur...,7.0,1237.0,1362.0,125.0,0.0,Michigan,NaN,NaN,...,0,0,0,0,0,0,1,0,1,Yes
3,Afton Alps,https://www.skiresort.info/ski-resort/afton-alps/,89.0,919.0,1270.0,351.0,5.0,Minnesota,300.0,60.0,...,0,12,0,0,0,0,2,2,19,No
4,Alpine Valley (Michigan),https://www.skiresort.info/ski-resort/alpine-v...,61.0,961.0,1211.0,249.0,4.0,Michigan,100.0,20.0,...,0,1,0,0,0,0,2,2,9,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435,Winterplace,https://www.skiresort.info/ski-resort/winterpl...,85.0,2995.0,3599.0,604.0,6.0,West Virginia,90.0,100.0,...,0,2,0,0,0,0,0,2,9,No
436,Wisp,https://www.skiresort.info/ski-resort/wisp-mch...,129.0,2415.0,3117.0,702.0,10.0,Maryland,132.0,100.0,...,0,0,0,0,0,0,1,5,13,No
437,Wolf Creek,https://www.skiresort.info/ski-resort/wolf-creek/,91.0,10299.0,11903.0,1604.0,62.0,Colorado,1600.0,430.0,...,0,1,0,0,0,1,0,2,11,No
438,Woods Valley,https://www.skiresort.info/ski-resort/woods-va...,65.0,574.0,1066.0,492.0,3.0,New York,25.0,180.0,...,0,2,0,0,0,0,0,2,5,No


### Finding resorts with missing price data

In [7]:
missing_price_rows = resort_info[resort_info['price'].isna()]
print(missing_price_rows[['name', 'state']])

                                             name          state
97                                 Chipmunk Hills          Idaho
101                                     Cloudmont        Alabama
106                                    Coppervale     California
124                                 Dynamite Hill       New York
137                               Frost Fire Park   North Dakota
143                                 Granite Gorge  New Hampshire
151                         Hard’ack (Aldis Hill)        Vermont
161                                  Hoedown Hill       Colorado
178  John & Melissa Besse Sports Park – Gladstone       Michigan
194                                Lee’s Ski Hill       Colorado
202                    Lost Trail Powder Mountain        Montana
220                                   Meany Lodge     Washington
222                            Missaukee Mountain       Michigan
239                                  Mount Eustis  New Hampshire
240                      

### Manually adding price data
I searched online for the 24/25 day ticket prices at each resort, most of them found on the resort's Facebook page. For Red Hill, I decided to use the price of membership even though it is a private ski area, since the membership is only $25 and open to anyone.

In [8]:
resort_info.loc[resort_info['name'] == 'Chipmunk Hills', 'price'] = 5.0
resort_info.loc[resort_info['name'] == 'Cloudmont', 'price'] = 40.0
resort_info.loc[resort_info['name'] == 'Coppervale', 'price'] = 40.0
resort_info.loc[resort_info['name'] == 'Dynamite Hill', 'price'] = 0.0
resort_info.loc[resort_info['name'] == 'Frost Fire Park', 'price'] = 50.0
resort_info.loc[resort_info['name'] == 'Granite Gorge', 'price'] = 55.0
resort_info.loc[resort_info['name'] == 'Hard’ack (Aldis Hill)', 'price'] = 0.0
resort_info.loc[resort_info['name'] == 'Hoedown Hill', 'price'] = 51.0
resort_info.loc[resort_info['name'] == 'John & Melissa Besse Sports Park – Gladstone', 'price'] = 12.0
resort_info.loc[resort_info['name'] == 'Lee’s Ski Hill', 'price'] = 0.0
resort_info.loc[resort_info['name'] == 'Lost Trail Powder Mountain', 'price'] = 68.0
resort_info.loc[resort_info['name'] == 'Meany Lodge', 'price'] = 60.0
resort_info.loc[resort_info['name'] == 'Missaukee Mountain', 'price'] = 10.0
resort_info.loc[resort_info['name'] == 'Mount Eustis', 'price'] = 0.0
resort_info.loc[resort_info['name'] == 'Mount Eyak', 'price'] = 30.0
resort_info.loc[resort_info['name'] == 'Nashoba Valley', 'price'] = 80.0
resort_info.loc[resort_info['name'] == 'Oglebay Resort', 'price'] = 34.0
resort_info.loc[resort_info['name'] == 'Paul Bunyan Ski Hill', 'price'] = 30.0
resort_info.loc[resort_info['name'] == 'Pinnacle Park', 'price'] = 20.0
resort_info.loc[resort_info['name'] == 'Red Hill – Moultonborough', 'price'] = 25.0 # Private but membership is $25 and open to anyone
resort_info.loc[resort_info['name'] == 'Terry Peak', 'price'] = 71.0
resort_info.loc[resort_info['name'] == 'Triangle Sports Area – Green Bay', 'price'] = 7.0
resort_info.loc[resort_info['name'] == 'Veterans Memorial – Franklin', 'price'] = 30.0

In [9]:
missing_price_rows = resort_info[resort_info['price'].isna()]
print(missing_price_rows[['name', 'state']])

Empty DataFrame
Columns: [name, state]
Index: []


### Filling in missing location data

In [10]:
missing_state_rows = resort_info[resort_info['state'].isna()]
print(missing_state_rows[['name', 'state']])

                            name state
119                 Diamond Peak   NaN
122             Donner Ski Ranch   NaN
145                 Granlibakken   NaN
153                     Heavenly   NaN
166     Homewood Mountain Resort   NaN
187                     Kirkwood   NaN
253                   Mount Rose   NaN
271  Northstar California Resort   NaN
281              Palisades Tahoe   NaN
324              Sierra at Tahoe   NaN
351                 Soda Springs   NaN
365                   Sugar Bowl   NaN
380                 Tahoe Donner   NaN


In [11]:
# Add state information for all Lake Tahoe resorts
resort_info.loc[resort_info['name'] == 'Diamond Peak', 'state'] = 'Nevada'
resort_info.loc[resort_info['name'] == 'Donner Ski Ranch', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Granlibakken', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Heavenly', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Homewood Mountain Resort', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Kirkwood', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Mount Rose', 'state'] = 'Nevada'
resort_info.loc[resort_info['name'] == 'Northstar California Resort', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Palisades Tahoe', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Sierra at Tahoe', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Soda Springs', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Sugar Bowl', 'state'] = 'California'
resort_info.loc[resort_info['name'] == 'Tahoe Donner', 'state'] = 'California'

In [12]:
missing_state_rows = resort_info[resort_info['state'].isna()]
print(missing_state_rows[['name', 'state']])

Empty DataFrame
Columns: [name, state]
Index: []


### Filling in missing elevation data

In [13]:
missing_elevation_rows = resort_info[resort_info['vertical_drop_ft'].isna()]
print(missing_elevation_rows[['name', 'state']])

               name          state
276  Oglebay Resort  West Virginia


In [14]:
resort_info.loc[resort_info['name'] == 'Oglebay Resort', 'vertical_drop_ft'] = 185.0
resort_info.loc[resort_info['name'] == 'Oglebay Resort', 'base_elevation_ft'] = 915.0
resort_info.loc[resort_info['name'] == 'Oglebay Resort', 'peak_elevation_ft'] = 1100.0

### Filling in missing slope length data

In [15]:
missing_length_rows = resort_info[resort_info['slope_mi'].isna()]
print(missing_length_rows[['name', 'state']])

                   name     state
326  Silverton Mountain  Colorado
377        Sunrise Park   Arizona


Since Silverton Mountain does not have any cut trails and is practically backcountry terrain, it doesn't really have a defined trail length. I also could not find a single source for the total trail length at Sunrise Park. Therefore I have to leave both of these blank.

### Fixing resorts with 0 miles of trails 
Because most sources only round to the nearest mile, there is not much available data for the total trail length of extremely small hills. There are many hills with only a few hundred feet of slopes and one surface lift, but these all get rounded down to 0. To fix this and prevent any issues with dividing by 0, I searched for any source of the length for each resort. If I couldn't find anything, I assigned a value of 0.1 for hills with only 1 trail and ~100 vertical feet and a value of 0.5 for slightly larger hills.

In [16]:
zero_length_rows = resort_info[resort_info['slope_mi'] == 0]
print(zero_length_rows[['name', 'state']])

                                             name           state
2                              Adventure Mountain        Michigan
30                                       Beartown        New York
92                                   Chapman Hill        Colorado
97                                 Chipmunk Hills           Idaho
101                                     Cloudmont         Alabama
124                                 Dynamite Hill        New York
135                                  Four Seasons        New York
136                         Frisco Adventure Park        Colorado
151                         Hard’ack (Aldis Hill)         Vermont
156                Hiawatha – Rock River Township        Michigan
161                                  Hoedown Hill        Colorado
178  John & Melissa Besse Sports Park – Gladstone        Michigan
183                                    Keyes Peak       Wisconsin
194                                Lee’s Ski Hill        Colorado
198       

In [17]:
resort_info.loc[resort_info['name'] == 'Adventure Mountain', 'slope_mi'] = 0.2
resort_info.loc[resort_info['name'] == 'Beartown', 'slope_mi'] = 0.5
resort_info.loc[resort_info['name'] == 'Chapman Hill', 'slope_mi'] = 0.4
resort_info.loc[resort_info['name'] == 'Chipmunk Hills', 'slope_mi'] = 0.3
resort_info.loc[resort_info['name'] == 'Cloudmont', 'slope_mi'] = 0.2
resort_info.loc[resort_info['name'] == 'Dynamite Hill', 'slope_mi'] = 0.1
resort_info.loc[resort_info['name'] == 'Four Seasons', 'slope_mi'] = 0.5
resort_info.loc[resort_info['name'] == 'Frisco Adventure Park', 'slope_mi'] = 0.1
resort_info.loc[resort_info['name'] == 'Hard’ack (Aldis Hill)', 'slope_mi'] = 0.1
resort_info.loc[resort_info['name'] == 'Hiawatha – Rock River Township', 'slope_mi'] = 0.3
resort_info.loc[resort_info['name'] == 'Hoedown Hill', 'slope_mi'] = 1.0
resort_info.loc[resort_info['name'] == 'John & Melissa Besse Sports Park – Gladstone', 'slope_mi'] = 0.4
resort_info.loc[resort_info['name'] == 'Keyes Peak', 'slope_mi'] = 0.3
resort_info.loc[resort_info['name'] == 'Lee’s Ski Hill', 'slope_mi'] = 0.1
resort_info.loc[resort_info['name'] == 'Living Memorial Park – Brattleboro', 'slope_mi'] = 0.3
resort_info.loc[resort_info['name'] == 'Mount McSauba', 'slope_mi'] = 0.4
resort_info.loc[resort_info['name'] == 'Pinnacle Park', 'slope_mi'] = 0.4
resort_info.loc[resort_info['name'] == 'Powderhouse Hill', 'slope_mi'] = 0.3
resort_info.loc[resort_info['name'] == 'Quoggy Joe Ski Center', 'slope_mi'] = 0.4
resort_info.loc[resort_info['name'] == 'Sapphire Valley', 'slope_mi'] = 0.5
resort_info.loc[resort_info['name'] == 'Triangle Sports Area – Green Bay', 'slope_mi'] = 0.2

In [18]:
zero_length_rows = resort_info[resort_info['slope_mi'] == 0]
print(zero_length_rows[['name', 'state']])

Empty DataFrame
Columns: [name, state]
Index: []


### Filling in missing skiable acreage data

In [19]:
missing_acreage_rows = resort_info[resort_info['skiable_acres'].isna()]
print(missing_acreage_rows[['name', 'state']])

                                 name          state
2                  Adventure Mountain       Michigan
41                          Big Moose          Maine
85                   Campton Mountain  New Hampshire
109                   Cranor Ski Hill       Colorado
133      Fort Wainwright – Birch Hill         Alaska
136             Frisco Adventure Park       Colorado
151             Hard’ack (Aldis Hill)        Vermont
156    Hiawatha – Rock River Township       Michigan
159                          Hillberg         Alaska
182                        Kettlebowl      Wisconsin
188                    Kissing Bridge       New York
199                     Lonesome Pine          Maine
239                      Mount Eustis  New Hampshire
270                  Northeast Slopes        Vermont
276                    Oglebay Resort  West Virginia
285              Paul Bunyan Ski Hill      Wisconsin
293                     Pinnacle Park          Maine
302                  Powderhouse Hill         

In [20]:
resort_info.loc[resort_info['name'] == 'Sun Valley: Dollar Mountain', 'skiable_acres'] = 189.0
resort_info.loc[resort_info['name'] == 'Rotarun', 'skiable_acres'] = 15.0

### Downloading final dataframe to CSV

In [21]:
resort_info.to_csv('ski_info.csv', index=False)